In [ ]:
# -----------------------------
# 1️⃣ Install required packages
# -----------------------------
!pip install -q streamlit requests PyPDF2 scikit-learn pandas
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# -----------------------------
# 2️⃣ Streamlit app code
# -----------------------------
app_code = """
import streamlit as st
import PyPDF2
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

st.set_page_config(page_title="AI Resume Screening System", layout="wide")
st.title("🤖 AI Resume Screening System")
st.markdown("### Upload Job Description and Resumes")

col1, col2 = st.columns(2)
with col1:
    jd = st.text_area("📄 Job Description", height=220)
with col2:
    resumes = st.file_uploader(
        "📑 Upload Resumes (PDF)",
        type=["pdf"],
        accept_multiple_files=True
    )

def extract_text(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\\s]", " ", text)
    text = re.sub(r"\\s+", " ", text).strip()
    return text

def extract_skills(jd_text, resume_text, top_n=6):
    jd_words = set(jd_text.split())
    resume_words = set(resume_text.split())
    matched = jd_words & resume_words
    common_words = {"and","or","the","with","using","for","to","in","a","of","on","at","is","as","by"}
    matched = list(matched - common_words)
    return matched[:top_n]

if st.button("🔍 Analyze Candidates"):
    if not jd or not resumes:
        st.warning("Please upload both Job Description and Resumes")
    else:
        st.success("✅ Analysis Completed")

        jd_text = preprocess(jd)
        resume_texts = []
        file_names = []

        for file in resumes:
            text = extract_text(file)
            text_clean = preprocess(text)
            resume_texts.append(text_clean)
            file_names.append(file.name)

        # TF-IDF Vectorization
        texts = [jd_text] + resume_texts
        vectorizer = TfidfVectorizer(stop_words="english")
        vectors = vectorizer.fit_transform(texts).toarray()
        jd_vector = vectors[0]
        resume_vectors = vectors[1:]

        candidate_scores = []
        for i, vec in enumerate(resume_vectors):
            score = cosine_similarity([jd_vector], [vec])[0][0] * 100
            matched_skills = extract_skills(jd_text, resume_texts[i], top_n=6)
            candidate_scores.append({
                "Candidate": file_names[i],
                "Match %": round(score, 2),
                "Top Skills": matched_skills
            })

        # Sort by Match %
        candidate_scores = sorted(candidate_scores, key=lambda x: x["Match %"], reverse=True)

        # Display Best Candidate badge
        best_candidate = candidate_scores[0]
        st.markdown(f"### 🏆 Best Candidate: {best_candidate['Candidate']} (Match %: {best_candidate['Match %']})")

        # Display candidates
        st.markdown("### 📊 Candidate Ranking with Top Skills")
        for c in candidate_scores:
            st.markdown(f"#### {c['Candidate']} (Match %: {c['Match %']})")
            st.markdown(f"**Top Skills:** " + ", ".join([f'`{s.upper()}`' for s in c['Top Skills']]))
            st.markdown("---")
"""

# -----------------------------
# 3️⃣ Write to app.py
# -----------------------------
with open("app.py", "w") as f:
    f.write(app_code)

# -----------------------------
# 4️⃣ Start Streamlit
# -----------------------------
import os, time
os.system("streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &")
time.sleep(5)

# -----------------------------
# 5️⃣ Start Cloudflare tunnel
# -----------------------------
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501


2025-12-23T15:21:48Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-23T15:21:48Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-23T15:21:51Z INF +--------------------------------------------------------------------------------------------+
2025-12-23T15:21:51Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-23T15:21:51Z INF |  https://tuning-regulations-pledge-talk.trycloudflare.